# Purpose
This file trains `num_optimizers` LSTM and GNN optimizers, use them to perform certain tasks, and then compare the statistics of their performance.

In [ ]:
import torch

from trainUtil.trainingLSTM import trainLSTM
from trainUtil.trainingGNN import trainGNN

In [ ]:
# Constants
num_optimizers = 10

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
# Trains optimizer
for i in range(num_optimizers):
    print(f"Training lstm-{i}")
    trainLSTM(3, f'trained_model/lstm_optimizer_{i}.pth')

    print(f"Training gnn-{i}")
    trainGNN(3, f'trained_model/gnn_optimizer_{i}.pth')

## Comparison